# Queries
## Resets
### Clear Unfinished
```sql
UPDATE `icml2021` SET host = null, executor = null, time_started = null, chosenmodel = null, errorrate = null, exception = null, time_end = null WHERE `time_started` is not null and time_end is null
```

### Clear Failed With `NoSuchFileException`
```sql
UPDATE `icml2021` SET host = null, executor = null, time_started = null, chosenmodel = null, errorrate = null, exception = null, time_end = null WHERE `exception` LIKE '%NoSuchFileException%'
```

### Clear auto-sklearn
```sql
UPDATE icml2021 SET host = null, executor = null, time_started = null, chosenmodel = null, errorrate = null, exception = null, time_end = null WHERE algorithm = "auto-sklearn"
```

```sql
UPDATE `icml2021` SET host = null, executor = null, time_started = null, chosenmodel = null, errorrate = null, exception = null, time_end = null WHERE (algorithm like '%python-scaling%' or algorithm like '%python-tuning%') and (`exception` LIKE '%NoSuchFileException%' or time_end is null)
```

## Progress
```sql
SELECT total, open, CONCAT(ROUND(100 * open / total, 2), "%") as "open (rel)", running, CONCAT(ROUND(100 * running / total, 2), "%") as "running (rel)", finished, CONCAT(ROUND(100 * finished / total, 2), "%") as "finished (rel)", successful, failed, CONCAT(ROUND(100 * successful / (successful + failed), 2), "%") as "success rate", CONCAT(ROUND(avgRuntimeFinished), "s") as "Average Time of Finished", CONCAT(ROUND(avgRuntimeFinished * open / running), "s") as "ETA" FROM (SELECT "aux" as pk, COUNT(*) as "open" FROM `icml2021` WHERE time_started is null) as t1 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as "running" FROM `icml2021` WHERE time_started is not null and time_end is null) as t2 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as finished, AVG(TIMESTAMPDIFF(SECOND, time_started, time_end)) as avgRuntimeFinished  FROM `icml2021` WHERE time_started is not null and time_end is not null) as t3 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as successful FROM `icml2021` where time_end is not null and exception is null) as t4 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as failed FROM `icml2021` where exception is not null) as t5 NATURAL JOIN (SELECT "aux" as pk, COUNT(*) as total FROM `icml2021`) as t6
```


## Select
- `SELECT algorithm, openmlid, count(*), avg(errorrate), avg(timestampdiff(second, time_started, time_end)) FROM icml2021 where time_end is not null group by algorithm,openmlid`
- `SELECT * FROM (SELECT openmlid, avg(errorrate) as "auto-sklearn", avg(timestampdiff(second, time_started, time_end)) FROM icml2021 where time_end is not null and algorithm = "auto-sklearn" group by openmlid) as t1 JOIN (SELECT openmlid, avg(errorrate) as "naive", avg(timestampdiff(second, time_started, time_end)) FROM icml2021 where time_end is not null and algorithm = "naive" group by openmlid) as t2 USING(openmlid)`



In [1]:
algos = ["naive-java-primitive", "naive-java-preprocessing"
         #, "naive-java-consolidated"
         ,"auto-sklearn",
        "naive-python-primitive",
        "naive-python-scaling",
        "naive-python-filtering",
        #"naive-python-wrapping",
        "naive-python-tuning"
        ]
query = "SELECT * FROM"
query += " NATURAL JOIN ".join([("(SELECT openmlid, avg(errorrate) as \"" + algo + "\", avg(timestampdiff(second, time_started, time_end)) as \"avg_runtime_" + algo + "\", COUNT(*) as \"n_" + algo + "\" FROM icml2021 where time_end is not null and algorithm = \"" + algo + "\" group by openmlid) as t" + str(i)) for i, algo in enumerate(algos)])
print(query)

SELECT * FROM(SELECT openmlid, avg(errorrate) as "naive-java-primitive", avg(timestampdiff(second, time_started, time_end)) as "avg_runtime_naive-java-primitive", COUNT(*) as "n_naive-java-primitive" FROM icml2021 where time_end is not null and algorithm = "naive-java-primitive" group by openmlid) as t0 NATURAL JOIN (SELECT openmlid, avg(errorrate) as "naive-java-preprocessing", avg(timestampdiff(second, time_started, time_end)) as "avg_runtime_naive-java-preprocessing", COUNT(*) as "n_naive-java-preprocessing" FROM icml2021 where time_end is not null and algorithm = "naive-java-preprocessing" group by openmlid) as t1 NATURAL JOIN (SELECT openmlid, avg(errorrate) as "auto-sklearn", avg(timestampdiff(second, time_started, time_end)) as "avg_runtime_auto-sklearn", COUNT(*) as "n_auto-sklearn" FROM icml2021 where time_end is not null and algorithm = "auto-sklearn" group by openmlid) as t2 NATURAL JOIN (SELECT openmlid, avg(errorrate) as "naive-python-primitive", avg(timestampdiff(second, 